In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pydicom as dicom
import pandas as pd
import numpy as np
from random import randint

import os, sys, time, shutil, scipy, cv2, json, datetime
import numpy as np
import pandas as pd
import PIL.Image
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm.notebook import tqdm

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

In [4]:
df = pd.read_csv('data/ISIC_2020_Training_GroundTruth_v2.csv')
df.columns = ['file', 'patient_id', 'lesion_id', 'gender', 'age', 'site', 'diagnosis', 'ben_mal', 'target']
df['jpg'] = df['file'].apply(lambda x: str(x) + ".jpg")
df.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target,jpg
0,ISIC_2637011,IP_7279968,IL_7972535,male,45.0,head/neck,unknown,benign,0,ISIC_2637011.jpg
1,ISIC_0015719,IP_3075186,IL_4649854,female,45.0,upper extremity,unknown,benign,0,ISIC_0015719.jpg
2,ISIC_0052212,IP_2842074,IL_9087444,female,50.0,lower extremity,nevus,benign,0,ISIC_0052212.jpg
3,ISIC_0068279,IP_6890425,IL_4255399,female,45.0,head/neck,unknown,benign,0,ISIC_0068279.jpg
4,ISIC_0074268,IP_8723313,IL_6898037,female,55.0,upper extremity,unknown,benign,0,ISIC_0074268.jpg


In [5]:
melanoma = df[df.target == 1]
non_melanoma = df[df.target == 0]
y_mel = melanoma.target
X_mel = melanoma.drop(columns=['target'], axis=1)
y_non_mel = non_melanoma.target
X_non_mel = non_melanoma.drop(columns=['target'], axis=1)

In [6]:
from sklearn.model_selection import train_test_split

X_tr_m, X_val_m, y_tr_m, y_val_m = train_test_split(X_mel, y_mel, random_state=42, test_size=0.2)
X_tr_nm, X_val_nm, y_tr_nm, y_val_nm = train_test_split(X_non_mel, y_non_mel, random_state=42, test_size=0.2)

mel_train = pd.concat([X_tr_m, y_tr_m], axis=1)
non_mel_train = pd.concat([X_tr_nm, y_tr_nm], axis=1)
mel_val = pd.concat([X_val_m, y_val_m], axis=1)
non_mel_val = pd.concat([X_val_nm, y_val_nm], axis=1)

train = pd.concat([mel_train, non_mel_train], axis=0)
val = pd.concat([mel_val, non_mel_val], axis=0)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

In [16]:
train.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,jpg,target
0,ISIC_1569119,IP_6120178,IL_3157235,male,65.0,torso,melanoma,malignant,ISIC_1569119.jpg,1
1,ISIC_7536704,IP_3994607,IL_1685088,male,75.0,torso,melanoma,malignant,ISIC_7536704.jpg,1
2,ISIC_3696488,IP_6051587,IL_8878499,female,90.0,upper extremity,melanoma,malignant,ISIC_3696488.jpg,1
3,ISIC_3343475,IP_4576785,IL_9262145,male,80.0,head/neck,melanoma,malignant,ISIC_3343475.jpg,1
4,ISIC_7785723,IP_8447624,IL_2711936,female,50.0,lower extremity,melanoma,malignant,ISIC_7785723.jpg,1


In [17]:
val.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,jpg,target
0,ISIC_6767569,IP_7665112,IL_1769376,male,70.0,head/neck,melanoma,malignant,ISIC_6767569.jpg,1
1,ISIC_7312977,IP_8675890,IL_2723500,male,55.0,lower extremity,melanoma,malignant,ISIC_7312977.jpg,1
2,ISIC_1785627,IP_0170821,IL_6029631,female,40.0,upper extremity,melanoma,malignant,ISIC_1785627.jpg,1
3,ISIC_8066110,IP_3055814,IL_7066374,male,50.0,lower extremity,melanoma,malignant,ISIC_8066110.jpg,1
4,ISIC_8838753,IP_6292815,IL_9486623,female,65.0,lower extremity,melanoma,malignant,ISIC_8838753.jpg,1


In [18]:
train.to_csv('data/train.csv', index=False)
val.to_csv('data/val.csv', index=False)

In [7]:
test1 = pd.read_csv('data/ISIC_2019_Training_GroundTruth.csv')
test2 = pd.read_csv('data/ISIC_2019_Training_Metadata.csv')
test1 = test1.rename(columns={'image':'image1'})
test2 = test2.rename(columns={'image':'image2'})
test = pd.concat([test2, test1], axis=1)
test = test.drop(columns=['image1'], axis=1)
test = test.drop(columns=['NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'], axis=1)
test.columns = ['file', 'age', 'site', 'lesion_id', 'gender', 'target']
test.target = test.target.astype(int)
test['jpg'] = test.file.apply(lambda x: str(x) + ".jpg")
test.head()

,file,age,site,lesion_id,gender,target,jpg
0,ISIC_0000000,55.0,anterior torso,NaN,female,0,ISIC_0000000.jpg
1,ISIC_0000001,30.0,anterior torso,NaN,female,0,ISIC_0000001.jpg
2,ISIC_0000002,60.0,upper extremity,NaN,female,1,ISIC_0000002.jpg
3,ISIC_0000003,30.0,upper extremity,NaN,male,0,ISIC_0000003.jpg
4,ISIC_0000004,80.0,posterior torso,NaN,male,1,ISIC_0000004.jpg


In [8]:
# assign folder variables
new_dir = 'split/'
tr_fldr = 'split/train'
tr_mel = 'split/train/mel2'
tt_fldr = 'split/test'
tt_mel = 'split/test/mel2'
images_dir = 'images/train_jpg'
test_dir = 'images/test'

In [19]:
mel_tr_ids = mel_train.jpg.to_list()

In [27]:
for img in tqdm(mel_tr_ids):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_mel, img))

  0%|          | 0/467 [00:00<?, ?it/s]

In [9]:
mel_val_ids = mel_val.jpg.to_list()
nm_val_ids = non_mel_val.jpg.to_list()

In [10]:
val_mel = 'split/val/mel2'
val_nm = 'split/val/not_mel2'

In [11]:
for img in tqdm(mel_val_ids):
    shutil.move(os.path.join(images_dir, img), os.path.join(val_mel, img))    

  0%|          | 0/117 [00:00<?, ?it/s]

In [12]:
for img in tqdm(nm_val_ids):
    shutil.move(os.path.join(images_dir, img), os.path.join(val_nm, img)) 

  0%|          | 0/6509 [00:00<?, ?it/s]

In [14]:
# move 
tt_mel_df = test[test.target == 1]
tt_nm_df = test[test.target == 0]
tt_mel_ids = tt_mel_df.jpg.to_list()
tt_nm_ids = tt_nm_df.jpg.to_list()


In [15]:
for img in tqdm(tt_mel_ids):
    shutil.move(os.path.join(test_dir, img), os.path.join(tt_mel, img))

  0%|          | 0/4522 [00:00<?, ?it/s]